In [ ]:
import os
import pandas as pd
import torch
from diffusers import StableDiffusionPipeline
from transformers import CLIPImageProcessor

csv_path = "midjourney_prompts_filtered.csv"
df = pd.read_csv(csv_path)

pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1")
device = torch.device("mps")
pipe.to(device)

image_folder = "generated_images"
os.makedirs(image_folder, exist_ok=True)

image_paths = []

for index, row in df.iterrows():
    
    if index > 30000:
        break

    prompt = row["text"]

    # generate images using the StableDiffusionPipeline
    generator = torch.Generator(device="cpu").manual_seed(0)  # Create generator on CPU
    image = pipe(prompt, height=768, width=768, guidance_scale=7.5, num_inference_steps=50, generator=generator).images[0]

    #image_tensor = image_processor(image, return_tensors="pt").pixel_values
    #image_tensor = image_tensor.squeeze(0)

    image_path = os.path.join(image_folder, f"{index}.png")
    image.save(image_path)
    image_paths.append(image_path)

df["image_path"] = image_paths
output_csv_path = "prompts_with_images.csv"
df.to_csv(output_csv_path, index=False)